In [1]:
import tensorflow as tf
from tensorflow.python import ipu

In [2]:
# Configure the IPU device.
config = ipu.config.IPUConfig()
config.auto_select_ipus = 2
config.configure_ipu_system()

In [3]:
# Create a simple model.
def create_model():
  return tf.keras.Sequential([
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
###end of def:

In [4]:
# Create a dataset for the model.
def create_dataset():
  mnist = tf.keras.datasets.mnist

  (x_train, y_train), (_, _) = mnist.load_data()
  x_train = x_train / 255.0

  train_ds = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(10000).batch(32, drop_remainder=True)
  train_ds = train_ds.map(lambda d, l:
                          (tf.cast(d, tf.float32), tf.cast(l, tf.int32)))

  return train_ds.repeat().prefetch(16)
###end of def:

In [5]:
dataset = create_dataset()

In [6]:
# Create a strategy for execution on the IPU.
strategy = ipu.ipu_strategy.IPUStrategy()
with strategy.scope():
  # Create a Keras model inside the strategy.
  model = create_model()

  # Compile the model for training.
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.RMSprop(),
      metrics=["accuracy"],
      # Anything between 2 and `steps_per_epoch` could help here.
      steps_per_execution=50,
  )

  model.fit(dataset, epochs=2, steps_per_epoch=100)
###end of with:

print("\n\nJob Done!!")

Epoch 1/2
100/100 [==============================] - 16s 158ms/step - loss: 3.5635 - accuracy: 0.4607
Epoch 2/2
100/100 [==============================] - 0s 671us/step - loss: 2.2846 - accuracy: 0.3746


Job Done!!
